In [19]:
import requests


url = "https://xlab-open-source.oss-cn-beijing.aliyuncs.com/open_leaderboard/activity/company/chinese/202311.json"
response = requests.get(url)


if response.status_code == 200:
    raw_dict = response.json()
else:
    print("Error: Failed to retrieve data from the URL")
    exit()


len(raw_dict)

3

In [20]:
from pydantic import BaseModel
from typing import List


class CompanyItem(BaseModel):
    name: str
    issue_comment: str
    open_issue: str
    open_pull: str
    review_comment: str
    merged_pull: str


class CompanyData(BaseModel):
    item: CompanyItem
    rank: int
    value: float
    rankDelta: int
    valueDelta: float


class CompanyChinaMonth(BaseModel):
    type: str
    time: str
    data: List[CompanyData]  # Change from CompanyData to List[CompanyData]


company_china_month = CompanyChinaMonth.model_validate(raw_dict)

company_china_month.data[0].model_dump()


{'item': {'name': 'Huawei',
  'issue_comment': '27931',
  'open_issue': '5304',
  'open_pull': '4342',
  'review_comment': '907',
  'merged_pull': '39137'},
 'rank': 1,
 'value': 27089.6,
 'rankDelta': 0,
 'valueDelta': 9107.38}